In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

In [5]:
def plot_series(time, series, format="-", start=0, end=None):
    plt.plot(time[start:end], series[start:end], format)
    plt.xlabel("Time")
    plt.ylabel("Value")
    plt.grid(True)

    
def trend(time, slope=0):
    return slope * time


def seasonal_pattern(season_time):
    """Just an arbitrary pattern, you can change it if you wish"""
    return np.where(season_time < 0.4,
                    np.cos(season_time * 2 * np.pi),
                    1 / np.exp(3 * season_time))


def seasonality(time, period, amplitude=1, phase=0):
    """Repeats the same pattern at each period"""
    season_time = ((time + phase) % period) / period
    return amplitude * seasonal_pattern(season_time)


def add_noise(time, noise_level=1, seed=None):
    rnd = np.random.RandomState(seed)
    return rnd.randn(len(time)) * noise_level

In [6]:
time = np.arange(4*365 + 1, dtype='float32')
baseline = 10
series = trend(time, 0.1)
amp = 40
slope = 0.05
noise = 5

In [7]:
series = (baseline 
          + trend(time, slope) 
          + seasonality(time, period=365, amplitude=amp)
          + add_noise(time, noise, seed=None))

In [8]:
split_time = 1000
time_train = time[:split_time]
X_train = series[:split_time]
time_valid = time[split_time:]
X_valid = series[split_time:]

In [9]:
window = 20
batch = 32
buffer = 1000

In [10]:
def window_data(series, window, batch, shuffle_buffer):
    data = tf.data.Dataset.from_tensor_slices(series)
    data = data.window(window + 1, shift=1 , drop_remainder=True)
    data = data.flat_map(lambda x: x.batch(window + 1))
    data = data.shuffle(shuffle_buffer).map(lambda x: (x[:-1], x[-1]))
    data = data.batch(batch).prefetch(1)
    return data

In [11]:
data = window_data(X_train, window, batch, buffer)

In [13]:
mod = tf.keras.models.Sequential([
    tf.keras.layers.Dense(10, input_shape=[window], activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1)])
mod.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                210       
_________________________________________________________________
dense_1 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 331
Trainable params: 331
Non-trainable params: 0
_________________________________________________________________


In [15]:
mod.compile(loss='mse', 
            optimizer=tf.keras.optimizers.SGD(lr=1e-6, momentum=0.9))
mod.fit(data, epochs=100, verbose=0)

In [16]:
forecast = []
for time in range(len(series) - window):
    forecast.append(mod.predict(series[time:time + window][np.newaxis]))
    
forecast = forecast[split_time - window:]
results = np.array(forecast)[:, 0, 0]

In [ ]:
plt.figure(figsize=(10, 5))
plot_series(time_valid, X_valid)
plot_series(time_valid, result)

In [2]:
tf.keras.metrics.mean_absolute_error(X_valid, results).numpy()

In [ ]:
data = window_data(X_train, window, batch, shuffle_buffer)
mod =  tf.keras.models.Sequential([
    tf.keras.layers.Dense(10, input_shape=[window_size], activation="relu"), 
    tf.keras.layers.Dense(10, activation="relu"), 
    tf.keras.layers.Dense(1)])

In [ ]:
lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-8 * 10**(epoch / 20))
optimizer = tf.keras.optimizers.SGD(lr=1e-8, momentum=0.9)
mod.compile(loss='mse', optimizer=optimizer)
history = mod.fit(
    dataset, epochs=100, callbacks=[lr_scheduler], verbose=0)

In [ ]:
lrs = 1e-8 * (10**(np.arange(100) / 20))
plt.semilogx(lrs, history.history['loss'])
plt.axis(1e-8, 1e-3, 0, 300)

In [ ]:
window_size = 30
dataset = window_data(X_train, window, batch, shuffle_buffer)

mod = tf.keras.models.Sequential([
  tf.keras.layers.Dense(10, activation="relu", input_shape=[window]),
  tf.keras.layers.Dense(10, activation="relu"),
  tf.keras.layers.Dense(1)])

optimizer = tf.keras.optimizers.SGD(lr=8e-6, momentum=0.9)
mod.compile(loss="mse", optimizer=optimizer)
history = mod.fit(dataset, epochs=500, verbose=0)

In [ ]:
loss = history.history['loss']
epochs = range(len(loss))
plt.plot(epochs, loss, 'b', label='Training Loss')

In [ ]:
# Plot all but the first 10
loss = history.history['loss']
epochs = range(10, len(loss))
plot_loss = loss[10:]
print(plot_loss)
plt.plot(epochs, plot_loss, 'b', label='Training Loss')

In [ ]:
# Plot all but the first 10
loss = history.history['loss']
epochs = range(10, len(loss))
plot_loss = loss[10:]
print(plot_loss)
plt.plot(epochs, plot_loss, 'b', label='Training Loss')

In [ ]:
tf.keras.metrics.mean_absolute_error(x_valid, results).numpy()